In [1]:
!pip3 install langchain_community

In [2]:
pip install -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_community.llms import OllamaLLM

In [6]:
llm = Ollama(model="llama3")
llm.invoke("The first man on the moon was ...")

NameError: name 'Ollama' is not defined

In [7]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3")
response = llm.invoke("The second man on the moon was ...")
print(response)

Actually, there wasn't a "second man" on the moon. The first humans to walk on the moon were Neil Armstrong and Edwin "Buzz" Aldrin, who landed on July 20, 1969 as part of the Apollo 11 mission.

Armstrong stepped out of the lunar module Eagle and became the first person to set foot on the moon's surface. Aldrin joined him a few minutes later, and together they spent about two and a half hours exploring the moon's surface.

While there have been several manned missions to the moon since Apollo 11, no one else has walked on the moon yet. However, NASA plans to return humans to the moon by 2024 as part of its Artemis program, with the goal of establishing a sustainable presence on the lunar surface and eventually sending humans to Mars.


In [8]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

'Here is the list with categories added:\n\nTaxi Utrecht - Transportation, Ministerie van Justitie en Veiligheid - Taxes, Etos AMSTERDAM NLD - Shopping, Bistro Bar Amsterdam - Dining'

### Read transaction data

In [9]:
# Read the transactions_2022_2023_r.csv file 
import pandas as pd
df = pd.read_csv("transactions_2022_2023_r.csv")
df.head()

,Date,Expense/Income,Name / Description,Amount (EUR)
0,1/1/2022,Expense,Online Shopping,241.86
1,1/3/2022,Income,Bonus Payment,2979.06
2,1/4/2022,Expense,Dining Out,699.41
3,1/7/2022,Expense,Insurance Payment,898.56
4,1/10/2022,Expense,Gym Membership,568.77


In [10]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

29

In [11]:
unique_transactions[1:10]

array(['Bonus Payment', 'Dining Out', 'Insurance Payment',
       'Gym Membership', 'Stock Dividends', 'Entertainment', 'Tax Refund',
       'Coffee Shop', 'Electricity Bill'], dtype=object)

### Categorise bank transactions with Llama3

In [12]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 29]

In [13]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [14]:
# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam',
                        llm)

["Based on the expense names, I've added a category for each one:", '', '1. ISS Catering Services De Meern - Food', '2. Taxi Utrecht - Transportation', '3. Etos AMSTERDAM NLD - Shopping', '4. Bistro Bar Amsterdam - Entertainment']


,Transaction vs category,Transaction,Category
0,"Based on the expense names, I've added a categ...","Based on the expense names, I've added a categ...",None
1,,,None
2,1. ISS Catering Services De Meern - Food,1. ISS Catering Services De Meern,Food
3,2. Taxi Utrecht - Transportation,2. Taxi Utrecht,Transportation
4,3. Etos AMSTERDAM NLD - Shopping,3. Etos AMSTERDAM NLD,Shopping
5,4. Bistro Bar Amsterdam - Entertainment,4. Bistro Bar Amsterdam,Entertainment


In [15]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Here are the categorized expenses with categories less than 4 words:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. Online Shopping - Retail', '4. Bonus Payment - Income', '5. Dining Out - Food', '6. Insurance Payment - Insurance', '7. Gym Membership - Fitness', '8. Stock Dividends - Investment', '9. Entertainment - Leisure', '10. Tax Refund - Taxes', '11. Coffee Shop - Food', '12. Electricity Bill - Utilities', '13. Side Business Earnings - Income', '14. Gift Received - Gifts', '15. Water Bill - Utilities', '16. Rent Payment - Rent', '17. Travel Expenses - Travel', '18. Investment Returns - Investment', '19. Entertainment - Leisure', '20. Grocery Shopping - Retail', '21. Selling Paintings - Art', '22. Clothing Purchase - Retail', '23. Real Estate Rental Income - Property', '24. Phone Bill - Utilities', '25. Public Transport Ticket - Transit', '26. Medical Expenses - Health', '27. Scholarship - Education', '28. Streaming Service Subs

In [16]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Here are the categorized expenses with categor...,Here are the categorized expenses with categor...,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. Online Shopping - Retail,3. Online Shopping,Retail
5,4. Bonus Payment - Income,4. Bonus Payment,Income
6,5. Dining Out - Food,5. Dining Out,Food
7,6. Insurance Payment - Insurance,6. Insurance Payment,Insurance
8,7. Gym Membership - Fitness,7. Gym Membership,Fitness
9,8. Stock Dividends - Investment,8. Stock Dividends,Investment


In [50]:
# categories_df_all.to_csv("categories_df_all.csv", index=False)

In [17]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Sport', 'Retail', 'Income', 'Food',
       'Insurance', 'Fitness', 'Investment', 'Leisure', 'Taxes',
       'Utilities', 'Gifts', 'Rent', 'Travel', 'Art', 'Property',
       'Transit', 'Health', 'Education', 'Utility', 'Auto'], dtype=object)

In [18]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [19]:
categories_df_all

,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Online Shopping - Retail,3. Online Shopping,Retail
5,4. Bonus Payment - Income,4. Bonus Payment,Income
6,5. Dining Out - Food,5. Dining Out,Food and Drinks
7,6. Insurance Payment - Insurance,6. Insurance Payment,Insurance
8,7. Gym Membership - Fitness,7. Gym Membership,Fitness
9,8. Stock Dividends - Investment,8. Stock Dividends,Investment
10,9. Entertainment - Leisure,9. Entertainment,Leisure
11,10. Tax Refund - Taxes,10. Tax Refund,Taxes


In [20]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '', regex=True)
categories_df_all

C:\Users\suhaa\AppData\Local\Temp\ipykernel_29344\3959838756.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '', regex=True)


,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Online Shopping - Retail,Online Shopping,Retail
5,4. Bonus Payment - Income,Bonus Payment,Income
6,5. Dining Out - Food,Dining Out,Food and Drinks
7,6. Insurance Payment - Insurance,Insurance Payment,Insurance
8,7. Gym Membership - Fitness,Gym Membership,Fitness
9,8. Stock Dividends - Investment,Stock Dividends,Investment
10,9. Entertainment - Leisure,Entertainment,Leisure
11,10. Tax Refund - Taxes,Tax Refund,Taxes


In [21]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023_r.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Expense/Income,Name / Description,Amount (EUR),Transaction vs category,Transaction,Category
0,1/1/2022,Expense,Online Shopping,241.86,3. Online Shopping - Retail,Online Shopping,Retail
1,1/3/2022,Income,Bonus Payment,2979.06,4. Bonus Payment - Income,Bonus Payment,Income
2,1/4/2022,Expense,Dining Out,699.41,5. Dining Out - Food,Dining Out,Food and Drinks
3,1/7/2022,Expense,Insurance Payment,898.56,6. Insurance Payment - Insurance,Insurance Payment,Insurance
4,1/10/2022,Expense,Gym Membership,568.77,7. Gym Membership - Fitness,Gym Membership,Fitness
...,...,...,...,...,...,...,...
309,12/26/2023,Expense,Clothing Purchase,207.26,22. Clothing Purchase - Retail,Clothing Purchase,Retail
310,12/27/2023,Expense,Dining Out,260.54,5. Dining Out - Food,Dining Out,Food and Drinks
311,12/29/2023,Expense,Streaming Service Subscription,862.08,28. Streaming Service Subscription - Entertain...,Streaming Service Subscription,Entertainment
312,12/29/2023,Expense,Clothing Purchase,769.78,22. Clothing Purchase - Retail,Clothing Purchase,Retail


In [34]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)